In [1]:
import pandas as pd
import numpy as np

In [2]:
#Slices the data to select only those course attempts for the given course
def crs_select():
    print('The available courses are ' + ', '.join(df['COURSE'].sort_values().unique()))
    crs = input('Please enter which course you would like to analyze: ')
    
    while crs not in df['COURSE'].unique():
        crs = input('Please enter a valid course:')
        
    dfCourse = df[df['COURSE']==crs]    
    return dfCourse, crs

In [3]:
#Load student success dataset. This dataset is artificially generated based on actual student 
#success data so as to avoid any concerns over FERPA violations or other misuse.

df = pd.read_csv('../data/artificial_student_success_data.csv', low_memory = False)

df = df[['STRM', 'ID', 'SUBJECT_CD', 'CRSE_NUM', 'OFFICIAL_GRADE', 'HOUSE_INCOME_DESCR', 'Race', 'GENDER', 'TERM_AGE', 'FIRST_GENERATION', 'max_HS_GPA']]

In [4]:
#Create a single column which contains the entire course information

course = df.apply(lambda x: x['SUBJECT_CD']+str(x['CRSE_NUM']), axis = 1)

df.insert(3, 'COURSE', course)

df = df.drop(['SUBJECT_CD', 'CRSE_NUM'], axis = 1)

In [5]:
#Select which course we want to analyze

dfCrs, course = crs_select()

The available courses are MATH10, MATH100, MATH104, MATH110, MATH120, MATH125, MATH129, MATH131, MATH132, MATH133, MATH145, MATH25, MATH300, MATH310, MATH32, MATH320, MATH330, MATH340, MATH342, MATH355, MATH356, MATH370, MATH372, MATH373, MATH400, MATH401, MATH402, MATH41, MATH410, MATH42, MATH420, MATHS45, MATHS72, MATHS73, MATHS95, STAT10, STAT105, STAT300, STAT305
Please enter which course you would like to analyze:STAT300


In [6]:
#Identify first attempts at the course and removes others

first_att = dfCrs.groupby(['ID'], sort = False)['STRM'].transform(min) == dfCrs['STRM']

dfCrs = dfCrs[first_att]

In [7]:
#Removes the occasional second attempt in the same term (this can happen if a student takes an 8 week course)

dfCrs = dfCrs.drop_duplicates(subset = ['ID', 'STRM'])

In [8]:
#Removes attempts which result in grade of EW

dfCrs = dfCrs[dfCrs['OFFICIAL_GRADE'] != 'EW']

In [9]:
#Replace NaN GPA rows with the mean

dfCrs['max_HS_GPA'].fillna(value=df['max_HS_GPA'].mean(), inplace=True)

In [10]:
#Pickle the data for future analysis

dfCrs.to_pickle('../data/'+course+'_clean.pkl')